In [2]:
# importing the libraries
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

In [3]:
# treading the csv file
df = pd.read_csv("diabetes.csv")
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [4]:
#checking the correation of the outcome colum with other
df.corr()["Outcome"]

Pregnancies                 0.221898
Glucose                     0.466581
BloodPressure               0.065068
SkinThickness               0.074752
Insulin                     0.130548
BMI                         0.292695
DiabetesPedigreeFunction    0.173844
Age                         0.238356
Outcome                     1.000000
Name: Outcome, dtype: float64

In [5]:
#data split
X = df.iloc[:,:-1].values
Y = df.iloc[:,-1].values

In [6]:
#scaling the dataset
from sklearn.preprocessing import StandardScaler
scaller = StandardScaler()

#scaling the data
X = scaller.fit_transform(X)

In [7]:
X.shape

(768, 8)

In [8]:
# spliting the dataset into train test split
from sklearn.model_selection import train_test_split
Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, Y, test_size=0.2, random_state=42)
    

In [9]:
print(f"the length of train data is {len(Xtrain)}")
print(f"The length of the test data is {len(Xtest)}")

the length of train data is 614
The length of the test data is 154


In [10]:
#building the model
import tensorflow
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense

In [ ]:
model  = Sequential()

model.add(Dense(32, activation='relu', input_dim = 8))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='Adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
output = model.fit(Xtrain, Ytrain, batch_size=32, epochs=100, validation_data=(Xtest, Ytest))

In [ ]:
plt.plot(output.history["accuracy"])
plt.legend(["train accuracy"])

plt.xlabel("epochs")
plt.ylabel("accuracy")
plt.show()

In [ ]:
plt.plot(output.history["val_accuracy"])
plt.legend(["val accuracy"])

plt.xlabel("epochs")
plt.ylabel("accuracy")
plt.show()

In [11]:
import kerastuner as kt

/var/folders/xb/sshzwv1128l88chp71np05t40000gn/T/ipykernel_32277/1654478174.py:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  import kerastuner as kt


### selecting the right optimizer

In [26]:
def build_model(hp):
    model = Sequential()

    model.add(Dense(32, activation='relu', input_dim = 8))
    model.add(Dense(1, activation='sigmoid'))

    optimizer_options = hp.Choice("optimizer", values = ["adam", "sgd", "rmsprop", "adadelta"])
    
    model.compile(optimizer= optimizer_options, loss = 'binary_crossentropy', metrics=["accuracy"])

    return model

In [ ]:
#creating a tunner object
tunner = kt.RandomSearch(build_model,
    objective = "val_accuracy", 
    max_trials = 5
)

In [28]:
tunner.search(Xtrain, Ytrain, epochs=5, validation_data = (Xtest, Ytest))

In [ ]:
tunner.get_best_hyperparameters()[0].values

In [ ]:
model = tunner.get_best_models(num_models=1)[0]

In [ ]:
model.summary()

In [ ]:
output = model.fit(Xtrain, Ytrain, epochs=100, initial_epoch=6, validation_data=(Xtest, Ytest))

In [ ]:
plt.plot(output.history["accuracy"])
plt.legend(["train accuracy"])

plt.xlabel("epochs")
plt.ylabel("accuracy")
plt.show()

### Selecting the number of nodes

In [20]:
def buildmodel_node(hp):
    model  = Sequential()

    units = hp.Int("units", 8, 128, step=8) #chenge nmber of nurons from 8 to 128 with step of 8

    model.add(Dense(units= units, activation='relu', input_dim = 8))
    model.add(Dense(units=1, activation = "sigmoid"))

    model.compile(optimizer='rmsprop', loss="binary_crossentropy", metrics=["accuracy"])

    return model


In [21]:
tunner_node = kt.RandomSearch(
    buildmodel_node,
    objective="val_accuracy",
    max_trials=5,
    directory = "vivek_1"
)

In [22]:
tunner_node.search(Xtrain, Ytrain, epochs=5, validation_data=(Xtest, Ytest))

Trial 5 Complete [00h 00m 02s]
val_accuracy: 0.7662337422370911

Best val_accuracy So Far: 0.7727272510528564
Total elapsed time: 00h 00m 08s


In [23]:
tunner_node.get_best_hyperparameters()[0].values

{'units': 64}

In [31]:
tunner_node.results_summary()

Results summary
Results in vivek_1/untitled_project
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 3 summary
Hyperparameters:
units: 64
Score: 0.7727272510528564

Trial 0 summary
Hyperparameters:
units: 56
Score: 0.7662337422370911

Trial 4 summary
Hyperparameters:
units: 104
Score: 0.7662337422370911

Trial 1 summary
Hyperparameters:
units: 24
Score: 0.701298713684082

Trial 2 summary
Hyperparameters:
units: 8
Score: 0.6103895902633667


In [24]:
model_node = tunner_node.get_best_models(num_models=1)[0]

/opt/miniconda3/envs/tensorflow/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/opt/miniconda3/envs/tensorflow/lib/python3.10/site-packages/keras/src/saving/saving_lib.py:576: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 6 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [25]:
model_node.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │           576 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 641 (2.50 KB)

 Trainable params: 641 (2.50 KB)

 Non-trainable params: 0 (0.00 B)

In [32]:
model_node.fit(Xtrain, Ytrain, epochs=100, initial_epoch=6, validation_data=(Xtest, Ytest))

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 0.7678 - loss: 0.4891 - val_accuracy: 0.7792 - val_loss: 0.5031
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7697 - loss: 0.4843 - val_accuracy: 0.7857 - val_loss: 0.4987
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7710 - loss: 0.4887 - val_accuracy: 0.7792 - val_loss: 0.4969
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7604 - loss: 0.4679 - val_accuracy: 0.7727 - val_loss: 0.4971
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7748 - loss: 0.4569 - val_accuracy: 0.7727 - val_loss: 0.4971
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7848 - loss: 0.4544 - val_accuracy: 0.7727 - val_loss: 0.4965
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7839 - loss: 0.4584 - val_accuracy: 0.7662 - val_loss: 0.4980
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7700 - loss: 0.4457 - val_accuracy: 0.76

### How to select the number of layers

### creating the model